In [15]:
import numpy as np

import numexpr
print(numexpr.detect_number_of_cores())
numexpr.set_num_threads(2)

from numba import jit, njit

8


In [16]:
N=10000000

In [17]:
a=np.random.rand(N)
b=np.random.rand(N)


Re=4000*np.exp(np.log(1e8/4000)*np.random.rand(N))
epsD=1e-6*np.exp(np.log(0.05/1e-6)*np.random.rand(N))


In [18]:
y1=np.zeros(len(Re),dtype='float')
y2=y1[:]
z=y1[:]


def func_CW30(Re,epsD):
    z=-8.6*np.ones(len(a))
    y1=epsD/3.7
    y2=(-2/np.log(10)*2.51)/Re

    for i in range(6):
        z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))

    z=(-2/np.log(10))*z
    return 1/(z*z)

%timeit func_CW30(Re,epsD)
z_=func_CW30(Re,epsD)

#AMD 3500u:
#5.55 s ± 279 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#Intel core 2 duo p8600? 
#15.4 s ± 1.92 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

#3610qm
#5.76 s ± 289 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

5.57 s ± 94.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
AA=1/np.sqrt(z_)+2/np.log(10)*np.log(epsD/3.7+2.51/(Re*np.sqrt(z_)))

In [20]:
max(abs(AA))*100

3.552713678800501e-13

In [21]:
def func_CW5(Re,epsD):

    z=-8.6*np.ones(len(Re))
    y1=epsD/3.7
    y2=(-2/np.log(10)*2.51)/Re


    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=(-2/np.log(10))*z
    return 1/(z*z)

%timeit func_CW5(Re,epsD)
z_hat=func_CW5(Re,epsD)


print('max abs rel err =',max(abs((z_-z_hat)/z_))*100,'%')

# AMD 3500u
# 1.1 s ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012977203796425436 %

#Intel core 2 duo
# 3.06 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012976754921686231 %

#3610qm
#1.1 s ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#max abs rel err = 0.01297598255585404 %

1.25 s ± 67.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
max abs rel err = 0.012977505889940028 %


In [22]:
max(Re)

99999839.3877693

In [23]:
def func_CW5_numexpr(Re,epsD):
    oo=np.ones(len(Re))
    z=numexpr.evaluate('-8.6*oo')
    y1=numexpr.evaluate('epsD/3.7')
    y2=numexpr.evaluate('(-2/log(10)*2.51)/Re')

    z=numexpr.evaluate('log(y1+y2*log(y1+y2*log(y1+y2*log(y1+y2*log(y1+y2*z)))))')
    z=numexpr.evaluate('(-2/log(10))*z')
    return numexpr.evaluate('1/(z*z)')

%timeit func_CW5_numexpr(Re,epsD)
z_hat=func_CW5_numexpr(Re,epsD)


print('max abs rel err =',max(abs((z_-z_hat)/z_))*100,'%')

# AMD 3500u
# 347 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012977203796425436 %

#intel core 2 duo
# 877 ms ± 21.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012976754921686231 %

#3610qm
#507 ms ± 9.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#max abs rel err = 0.01297598255585404 %

573 ms ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
max abs rel err = 0.012977505889940028 %


In [24]:
@njit(parallel=True)
def func_CW5_numba(Re,epsD):

    z=-8.6*np.ones(len(Re))
    y1=epsD/3.7
    y2=(-2/np.log(10)*2.51)/Re


    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=(-2/np.log(10))*z
    return 1/(z*z)

%timeit func_CW5_numba(Re,epsD)
z_hat=func_CW5_numba(Re,epsD)


print('max abs rel err =',max(abs((z_-z_hat)/z_))*100,'%')

# AMD 3500u
# 121 ms ± 7.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012977203796425436 %

#Intel core 2 duo
# 566 ms ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# max abs rel err = 0.012976754921686231 %

#3610qm
#81.9 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#max abs rel err = 0.01297598255585404 %

82.1 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
max abs rel err = 0.012977505889940028 %
